In [ ]:
!pip install -q ultralytics
!pip install -q tools
!pip install --upgrade pymupdf
import fitz

In [ ]:
import os
import json

# If needed, install PyMuPDF in Colab:
# !pip install --upgrade pymupdf

import fitz  # from pymupdf
from PIL import Image, ImageDraw, ImageFont

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files_path = "test_files"
output_json_path = "output_json/annotations.json"
save_dir = "images_with_boxes"



In [ ]:
import os
import json

import fitz              # PyMuPDF
import numpy as np
import cv2


def visualize_pdf_annotations_cv(pdf_dir, json_path, out_dir):
    os.makedirs(out_dir, exist_ok=True)

    # Load annotation JSON
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    for pdf_name, pages in data.items():
        pdf_path = os.path.join(pdf_dir, pdf_name)
        if not os.path.exists(pdf_path):
            print(f"WARNING: PDF not found: {pdf_path}, skipping.")
            continue

        try:
            doc = fitz.open(pdf_path)
        except Exception as e:
            print(f"ERROR opening {pdf_path}: {e}")
            continue

        pdf_stem, _ = os.path.splitext(pdf_name)

        for page_key, page_data in pages.items():
            # page_key format: "page_3"
            try:
                page_num = int(page_key.split("_")[1])
            except Exception:
                print(f"WARNING: cannot parse page number from {page_key}")
                continue

            page_index = page_num - 1
            if page_index < 0 or page_index >= len(doc):
                print(f"WARNING: page index {page_index} out of range for {pdf_name}")
                continue

            page = doc[page_index]

            # Target size from JSON
            target_w = page_data["page_size"]["width"]
            target_h = page_data["page_size"]["height"]

            rect = page.rect
            scale_x = target_w / rect.width
            scale_y = target_h / rect.height
            matrix = fitz.Matrix(scale_x, scale_y)

            pix = page.get_pixmap(matrix=matrix)

            # Convert pixmap to NumPy array (RGB), and make it WRITABLE
            img = np.frombuffer(pix.samples, dtype=np.uint8)
            img = img.reshape(pix.height, pix.width, pix.n).copy()  # copy() makes it writable

            # Handle channels
            if pix.n == 4:
                # RGBA -> RGB
                img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
            elif pix.n == 1:
                # grayscale -> RGB
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            # Convert RGB -> BGR for OpenCV drawing
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

            # Draw annotations: boxes + labels
            for ann in page_data.get("annotations", []):
                inner = next(iter(ann.values()))
                category = str(inner["category"])
                bbox = inner["bbox"]

                x = float(bbox["x"])
                y = float(bbox["y"])
                w = float(bbox["width"])
                h = float(bbox["height"])

                x1, y1 = int(round(x)), int(round(y))
                x2, y2 = int(round(x + w)), int(round(y + h))

                # Red rectangle (BGR: (0, 0, 255))
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # Green label text slightly above or inside the box
                text = category
                text_x = max(0, x1)
                text_y = max(20, y1 - 5)  # ensure inside image

                cv2.putText(
                    img,
                    text,
                    (text_x, text_y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2.0,               # font scale
                    (0, 255, 0),       # green text
                    4,
                    cv2.LINE_AA,
                )

            # Save file as PNG
            out_name = f"{pdf_stem}_page_{page_num}_annotated.png"
            out_path = os.path.join(out_dir, out_name)
            cv2.imwrite(out_path, img)
            print(f"Saved: {out_path}")

        doc.close()


In [ ]:
visualize_pdf_annotations_cv(files_path, output_json_path, save_dir)

Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-2_page_1_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-2_page_2_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-2_page_3_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-2_page_4_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-2_page_5_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-41-_page_1_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-41-_page_2_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-41-_page_3_annotated.png
Saved: /content/drive/MyDrive/HACKATHON_ARMETA/predicted_with_boxes/ft/Паспортпроекта-41-_page_4_annotated.png
Saved: /con